<a href="https://colab.research.google.com/github/kuzminadya/FHP-infovis-py/blob/master/Data_Dossier_%7C_Repo_Analysis.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Font File Transformations, the 1st Use Case

## An idea

My idea is to analyse and visualise the transformation of font file changing over time. Creating a font, especially a large font family may take several years of work or it might be a result of collaboration of several designers and type engineers. Even after the first version is released, the contribution to the project and updating the font may last for several years.


### The first use case — analysis of a existed github repo

As an examle I use IBM Plex® typeface which was active on Github for a 2 years and have more than 219 commits:\
https://github.com/IBM/plex 

First of all I'm going to import our favorite libraries.


In [2]:
import pandas as pd
import altair as alt

The first step is getting the dataset in JSON format from Github (https://developer.github.com/v3/repos/#get-a-repository
). Here I have a description of repo itself.

In [3]:
import requests 

url = ('https://api.github.com/repos/IBM/plex')

response = requests.get(url)

print(f"Request returned {response.status_code} : '{response.reason}'")
dataset_repo_json = response.json()

# dataset_repo_json

Request returned 200 : 'OK'


And then I get the list of commits (https://developer.github.com/v3/repos/commits/) in JSON format:

In [4]:
url = ('https://api.github.com/repos/IBM/plex/commits')

response = requests.get(url)

print(f"Request returned {response.status_code} : '{response.reason}'")
dataset_commits_json = response.json()

# dataset_commits_json

Request returned 200 : 'OK'


The next part is converting JSON into a dataframe. I will convert only the second file.

With a help of: https://stackoverflow.com/questions/41168558/python-how-to-convert-json-file-to-dataframe

In [5]:
from pandas.io.json import json_normalize
df_commits = pd.DataFrame.from_dict(json_normalize(dataset_commits_json), orient='columns')

/usr/local/lib/python3.6/dist-packages/ipykernel_launcher.py:2: FutureWarning: pandas.io.json.json_normalize is deprecated, use pandas.json_normalize instead
  


In [6]:
df_commits.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 30 entries, 0 to 29
Data columns (total 59 columns):
 #   Column                         Non-Null Count  Dtype  
---  ------                         --------------  -----  
 0   sha                            30 non-null     object 
 1   node_id                        30 non-null     object 
 2   url                            30 non-null     object 
 3   html_url                       30 non-null     object 
 4   comments_url                   30 non-null     object 
 5   parents                        30 non-null     object 
 6   commit.author.name             30 non-null     object 
 7   commit.author.email            30 non-null     object 
 8   commit.author.date             30 non-null     object 
 9   commit.committer.name          30 non-null     object 
 10  commit.committer.email         30 non-null     object 
 11  commit.committer.date          30 non-null     object 
 12  commit.message                 30 non-null     objec

Some operations with df, like decreasing the amount of columns, haven't succeed with json file. To avoid it I transform the file in csv format and download it (to do it please uncomment the code below).

In [7]:
from google.colab import files

df_commits.to_csv('df_commits_table.csv')
# files.download('df_commits_table.csv')

<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>

In [8]:
file_commits = open("df_commits_table.csv")

file_commits_csv = pd.read_csv(file_commits, sep=',', keep_default_na=False)
file_commits_csv.head()

,Unnamed: 0,sha,node_id,url,html_url,comments_url,parents,commit.author.name,commit.author.email,commit.author.date,commit.committer.name,commit.committer.email,commit.committer.date,commit.message,commit.tree.sha,commit.tree.url,commit.url,commit.comment_count,commit.verification.verified,commit.verification.reason,commit.verification.signature,commit.verification.payload,author.login,author.id,author.node_id,author.avatar_url,author.gravatar_id,author.url,author.html_url,author.followers_url,author.following_url,author.gists_url,author.starred_url,author.subscriptions_url,author.organizations_url,author.repos_url,author.events_url,author.received_events_url,author.type,author.site_admin,committer.login,committer.id,committer.node_id,committer.avatar_url,committer.gravatar_id,committer.url,committer.html_url,committer.followers_url,committer.following_url,committer.gists_url,committer.starred_url,committer.subscriptions_url,committer.organizations_url,committer.repos_url,committer.events_url,committer.received_events_url,committer.type,committer.site_admin,author,committer
0,0,3fbdb5dd75d74379b7fba47dd889a1eb70771d73,MDY6Q29tbWl0MTA1NzAyNjMxOjNmYmRiNWRkNzVkNzQzNz...,https://api.github.com/repos/IBM/plex/commits/...,https://github.com/IBM/plex/commit/3fbdb5dd75d...,https://api.github.com/repos/IBM/plex/commits/...,[{'sha': '968081d61b6abe18cd0dc14a42704afdf483...,Vince Picone,vpicone@gmail.com,2020-06-30T23:58:27Z,GitHub,noreply@github.com,2020-06-30T23:58:27Z,Merge pull request #323 from dtinth/patch-1\n\...,099315db66b26edb264f5b68047ac8ebff714a25,https://api.github.com/repos/IBM/plex/git/tree...,https://api.github.com/repos/IBM/plex/git/comm...,0,True,valid,-----BEGIN PGP SIGNATURE-----\n\nwsBcBAABCAAQB...,tree 099315db66b26edb264f5b68047ac8ebff714a25\...,vpicone,4078018.0,MDQ6VXNlcjQwNzgwMTg=,https://avatars1.githubusercontent.com/u/40780...,,https://api.github.com/users/vpicone,https://github.com/vpicone,https://api.github.com/users/vpicone/followers,https://api.github.com/users/vpicone/following...,https://api.github.com/users/vpicone/gists{/gi...,https://api.github.com/users/vpicone/starred{/...,https://api.github.com/users/vpicone/subscript...,https://api.github.com/users/vpicone/orgs,https://api.github.com/users/vpicone/repos,https://api.github.com/users/vpicone/events{/p...,https://api.github.com/users/vpicone/received_...,User,False,web-flow,19864447.0,MDQ6VXNlcjE5ODY0NDQ3,https://avatars3.githubusercontent.com/u/19864...,,https://api.github.com/users/web-flow,https://github.com/web-flow,https://api.github.com/users/web-flow/followers,https://api.github.com/users/web-flow/followin...,https://api.github.com/users/web-flow/gists{/g...,https://api.github.com/users/web-flow/starred{...,https://api.github.com/users/web-flow/subscrip...,https://api.github.com/users/web-flow/orgs,https://api.github.com/users/web-flow/repos,https://api.github.com/users/web-flow/events{/...,https://api.github.com/users/web-flow/received...,User,False,,
1,1,f90d92225eda9e0056d13446cc979a83e11bd0a1,MDY6Q29tbWl0MTA1NzAyNjMxOmY5MGQ5MjIyNWVkYTllMD...,https://api.github.com/repos/IBM/plex/commits/...,https://github.com/IBM/plex/commit/f90d92225ed...,https://api.github.com/repos/IBM/plex/commits/...,[{'sha': '968081d61b6abe18cd0dc14a42704afdf483...,Thai Pangsakulyanont,dtinth@spacet.me,2020-06-21T12:56:02Z,GitHub,noreply@github.com,2020-06-21T12:56:02Z,"Add ""main"" field to ""package.json""\n\nThis all...",099315db66b26edb264f5b68047ac8ebff714a25,https://api.github.com/repos/IBM/plex/git/tree...,https://api.github.com/repos/IBM/plex/git/comm...,0,True,valid,-----BEGIN PGP SIGNATURE-----\n\nwsBcBAABCAAQB...,tree 099315db66b26edb264f5b68047ac8ebff714a25\...,dtinth,193136.0,MDQ6VXNlcjE5MzEzNg==,https://avatars3.githubusercontent.com/u/19313...,,https://api.github.com/users/dtinth,https://github.com/dtinth,https://api.github.com/users/dtinth/followers,https://api.github.com/users/dtinth/following{...,https://api.github.com/users/dtinth/gists{/gis...,h

In [9]:
file_commits_csv.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 30 entries, 0 to 29
Data columns (total 60 columns):
 #   Column                         Non-Null Count  Dtype 
---  ------                         --------------  ----- 
 0   Unnamed: 0                     30 non-null     int64 
 1   sha                            30 non-null     object
 2   node_id                        30 non-null     object
 3   url                            30 non-null     object
 4   html_url                       30 non-null     object
 5   comments_url                   30 non-null     object
 6   parents                        30 non-null     object
 7   commit.author.name             30 non-null     object
 8   commit.author.email            30 non-null     object
 9   commit.author.date             30 non-null     object
 10  commit.committer.name          30 non-null     object
 11  commit.committer.email         30 non-null     object
 12  commit.committer.date          30 non-null     object
 13  commit.

Converting a column with time-based data

In [10]:
file_commits_csv['commit.author.date'] = pd.to_datetime(file_commits_csv['commit.author.date'], format='%Y-%m-%d')

Converting commit.author.name column to string

In [26]:
#  file_commits_csv['commit.author.name'] = file_commits_csv['commit.author.name'].astype('str')
# file_commits_csv['commit.author.name.str'] = file_commits_csv['commit.author.name'].str.split(',') 

In [27]:
df_commits_reduced = file_commits_csv[["commit.author.name", "commit.author.date", "commit.message"]]
df_commits_reduced.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 30 entries, 0 to 29
Data columns (total 3 columns):
 #   Column              Non-Null Count  Dtype              
---  ------              --------------  -----              
 0   commit.author.name  30 non-null     object             
 1   commit.author.date  30 non-null     datetime64[ns, UTC]
 2   commit.message      30 non-null     object             
dtypes: datetime64[ns, UTC](1), object(2)
memory usage: 848.0+ bytes


Renaming the columns.\
(Thank you, Marian!) 

In [28]:
df_commits_reduced.columns = ["commit-author-name", "commit-author-date", "commit-message"]

Adding a column with an int value in order to count the amount of items. Prabably a cheap trick


In [30]:
df_commits_reduced['value'] = 1

/usr/local/lib/python3.6/dist-packages/ipykernel_launcher.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  """Entry point for launching an IPython kernel.


In [31]:
df_commits_reduced.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 30 entries, 0 to 29
Data columns (total 4 columns):
 #   Column              Non-Null Count  Dtype              
---  ------              --------------  -----              
 0   commit-author-name  30 non-null     object             
 1   commit-author-date  30 non-null     datetime64[ns, UTC]
 2   commit-message      30 non-null     object             
 3   value               30 non-null     int64              
dtypes: datetime64[ns, UTC](1), int64(1), object(2)
memory usage: 1.1+ KB


In [33]:
# list(df_commits_reduced.values)

In [34]:
df_repo_groupby_name = df_commits_reduced.groupby('commit-author-name').count()
df_repo_groupby_name

,commit-author-date,commit-message,value
commit-author-name,,,
LMapes,1,1,1
Mike Abbink,1,1,1
Thai Pangsakulyanont,1,1,1
Vince Picone,27,27,27


In [17]:
# import numpy as np
# import matplotlib.pyplot as plt

# df_commits_reduced.plot.scatter(x='commit.author.date', y='value')

## List of broken visualisations :(



In [35]:
alt.Chart(df_commits_reduced).mark_area(interpolate='monotone',
    fillOpacity=0.8,
    stroke='white',
    strokeWidth=0.2).encode(
    alt.X('monthdate(commit-author-date):T',
        axis=alt.Axis(format='%M', domain=False, tickSize=0)
    ),
    alt.Y('sum(value):Q', stack='center', axis=None),
    alt.Color('commit-author-name:N',
        scale=alt.Scale(scheme='category20b')
    )
# ).transform_aggregate(
#     # sum_val='sum(value)',
#     # groupby=['commit.author.name']
).properties(
    width=800,
    height=400
).interactive()

alt.Chart(...)

In [37]:
alt.Chart(df_commits_reduced).mark_circle(size=50).encode(
    alt.Y('hoursminutes(commit-author-date):T', title='hour of day'),
    alt.X('monthdate(commit-author-date):T', title='date'),
    alt.Color('commit-author-name:N')
).interactive()

alt.Chart(...)

In [38]:
alt.Chart(df_commits_reduced).mark_bar().encode(
    x='commit-author-name:O',
    y='count_val:Q'
).transform_aggregate(
    count_val='count(value)',
    groupby=['commit-author-name']
)

alt.Chart(...)

In [39]:
df_commits_reduced = df_commits_reduced.set_index("commit-author-date")

In [40]:
days = df_commits_reduced.resample("D").sum()

In [41]:
days

,value
commit-author-date,
2019-10-03 00:00:00+00:00,3
2019-10-04 00:00:00+00:00,1
2019-10-05 00:00:00+00:00,2
2019-10-06 00:00:00+00:00,0
2019-10-07 00:00:00+00:00,0
...,...
2020-06-26 00:00:00+00:00,0
2020-06-27 00:00:00+00:00,0
2020-06-28 00:00:00+00:00,0


In [43]:
week = df_commits_reduced.resample("W").sum()
# week

In [42]:
# week.astype(int)
# week.info()

In [ ]:
# month = df_commits_reduced.resample("M").sum()
# month

In [44]:
alt.Chart(week.reset_index().melt("commit-author-date")).mark_line(opacity=0.75, interpolate="basis").encode(
    x='commit-author-date',
    y='value',
    color='variable',
).properties(width=800, height=400)

alt.Chart(...)